In [10]:
#subdir in results
result_path = 'results/simcyp298/'
#result_path = 'results/pmcsimcyp144/'


# load puptator file
#filelist = ['data/Tafamidis97_filtered.pubtator']
filelist = ['data/SimCyp298_filtered.pubtator']

# pmc
#filelist = ['data/pmc_sim_cyp_144_chemcials_diseases_filtered.pubtator']

In [ ]:
import os
print('remove snorkel db...')
os.remove("snorkel.db") 
print('snorkel db removed')

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

In [3]:
from snorkel import SnorkelSession
from snorkel.parser import CorpusParser, Spacy, StanfordCoreNLPServer
from pubtator import PubTatorDocPreprocessor, PubTatorTagProcessor, PubTatorParser
from time import time

parser = "spacy"
num_procs = 1

start_ts = time()


for fp in filelist:
    doc_preprocessor = PubTatorDocPreprocessor(fp, annotations=False, debug=True)
    #arser = Spacy() if parser == "spacy" else StanfordCoreNLPServer()
    parser = PubTatorParser(stop_on_err=False)
    corpus_parser = CorpusParser(parser=parser)
    corpus_parser.apply(doc_preprocessor, parallelism=num_procs, clear=True)
    end_ts = time()

print("\nDONE in {}".format((time() - start_ts)))

Clearing existing...
Running UDF...
DEBUG: processing doc_id: 22339447 ...
DEBUG: processing doc_id: 22348413 ...
DEBUG: processing doc_id: 22358108 ...
DEBUG: processing doc_id: 22451032 ...
DEBUG: processing doc_id: 22529023 ...
DEBUG: processing doc_id: 22686346 ...
DEBUG: processing doc_id: 22912565 ...
DEBUG: processing doc_id: 22990330 ...
DEBUG: processing doc_id: 23032911 ...
DEBUG: processing doc_id: 23105125 ...
DEBUG: processing doc_id: 23220613 ...
DEBUG: processing doc_id: 23252946 ...
DEBUG: processing doc_id: 23256625 ...
DEBUG: processing doc_id: 23263738 ...
DEBUG: processing doc_id: 23314529 ...
DEBUG: processing doc_id: 23334403 ...
DEBUG: processing doc_id: 23407140 ...
DEBUG: processing doc_id: 23469684 ...
DEBUG: processing doc_id: 23492463 ...
DEBUG: processing doc_id: 23588308 ...
DEBUG: processing doc_id: 23589735 ...
DEBUG: processing doc_id: 23698259 ...
DEBUG: processing doc_id: 23726291 ...
DEBUG: processing doc_id: 23732466 ...
DEBUG: processing doc_id: 23

DEBUG: processing doc_id: 10435174 ...
DEBUG: processing doc_id: 10460065 ...
DEBUG: processing doc_id: 10513779 ...
DEBUG: processing doc_id: 10709776 ...
DEBUG: processing doc_id: 10877724 ...
DEBUG: processing doc_id: 10934672 ...
DEBUG: processing doc_id: 10976543 ...
DEBUG: processing doc_id: 10984808 ...
DEBUG: processing doc_id: 11005703 ...
DEBUG: processing doc_id: 11029845 ...
DEBUG: processing doc_id: 11061578 ...
DEBUG: processing doc_id: 11075313 ...
DEBUG: processing doc_id: 11122720 ...
DEBUG: processing doc_id: 11126989 ...
DEBUG: processing doc_id: 11180018 ...
DEBUG: processing doc_id: 11181496 ...
DEBUG: processing doc_id: 11197581 ...
DEBUG: processing doc_id: 11259986 ...
DEBUG: processing doc_id: 11272871 ...
DEBUG: processing doc_id: 11422004 ...
DEBUG: processing doc_id: 11465417 ...
DEBUG: processing doc_id: 11471312 ...
DEBUG: processing doc_id: 11474784 ...
DEBUG: processing doc_id: 11475198 ...
DEBUG: processing doc_id: 11482929 ...
DEBUG: processing doc_id:

In [4]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

Documents: 298
Sentences: 3452


In [5]:
import os


try:  
    os.mkdir(result_path)
except OSError:  
    print ("Creation of the directory {} failed because it may exists".format(result_path))
else:  
    print ("Successfully created the directory %s " % result_path)


Creation of the directory results/simcyp298 failed because it may exists


In [6]:
from snorkel.models import Document

all_docs = session.query(Document).all()

print('Amount of docs: {}'.format(len(all_docs)))
with open(result_path + 'doc_mapping.tsv', 'w') as f:
    f.write('{}\t{}'.format('snorkel_id', 'pmid'))
    for doc in all_docs:
        f.write('\n{}\t{}'.format(doc.id, doc.name))
print('Finished')

Amount of docs: 298
Finished


In [7]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

print("Loading all sentences from db...")
all_sents = session.query(Sentence).all()
print("Loading complete!")


print('Amount of sentences: {}'.format(len(all_sents)))

Documents: 298
Sentences: 3452
Loading all sentences from db...
Loading complete!
Amount of sentences: 3452


In [ ]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])
candidate_extractor = PretaggedCandidateExtractor(ChemicalDisease, ['Chemical', 'Disease'])

for k, sents in enumerate([all_sents]):
    candidate_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(ChemicalDisease).filter(ChemicalDisease.split == k).count())
    

from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('chemical_disease.lstm')

print("Loading all candidates from db...")
all_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).order_by(ChemicalDisease.id).all()
print("{} candidates load from db!".format(len(all_cands)))


print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id', 'chemical_cid', 'chemical_span', 'disease_cid', 'disease_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "chemical_disease_association.tsv", session, all_cands, all_sents, header_str, 'chemical_cid', 'disease_cid')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

In [ ]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

ChemicalGeneInteraction = candidate_subclass('ChemicalGeneInteraction', ['chemical', 'gene'])
candidate_extractor = PretaggedCandidateExtractor(ChemicalGeneInteraction, ['Chemical', 'Gene'])

for k, sents in enumerate([all_sents]):
    candidate_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(ChemicalGeneInteraction).filter(ChemicalGeneInteraction.split == k).count())
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('chemical_gene_interaction.lstm')


print("Loading all candidates from db...")
all_cands = session.query(ChemicalGeneInteraction).filter(ChemicalGeneInteraction.split == 0).order_by(ChemicalGeneInteraction.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','chemical_cid', 'chemical_span', 'gene_cid', 'gene_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "chemical_gene_interaction.tsv", session, all_cands, all_sents, header_str, 'chemical_cid', 'gene_cid')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

In [ ]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

GeneDiseaseInteraction = candidate_subclass('GeneDiseaseInteraction', ['gene', 'disease'])
candidate_extractor = PretaggedCandidateExtractor(GeneDiseaseInteraction, ['Gene', 'Disease'])

for k, sents in enumerate([all_sents]):
    candidate_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(GeneDiseaseInteraction).filter(GeneDiseaseInteraction.split == k).count())
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('gene_disease_interaction.lstm')

print("Loading all candidates from db...")
all_cands = session.query(GeneDiseaseInteraction).filter(GeneDiseaseInteraction.split == 0).order_by(GeneDiseaseInteraction.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','gene_cid', 'gene_span', 'disease_cid', 'disease_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "gene_disease_interaction.tsv", session, all_cands, all_sents, header_str, 'gene_cid', 'disease_cid')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

In [11]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor


GeneChemicalMetabolism = candidate_subclass('GeneChemicalMetabolism', ['gene', 'chemical'])
candidate_gene_chemical_metabolism_extractor = PretaggedCandidateExtractor(GeneChemicalMetabolism, ['Gene', 'Chemical'])

for k, sents in enumerate([all_sents]):
    candidate_gene_chemical_metabolism_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(GeneChemicalMetabolism).filter(GeneChemicalMetabolism.split == k).count())
    
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('gene_chemical_metabolism.lstm')

print("Loading all candidates from db...")
all_cands = session.query(GeneChemicalMetabolism).filter(GeneChemicalMetabolism.split == 0).order_by(GeneChemicalMetabolism.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','gene_cid', 'gene_span', 'chemical_cid', 'chemical_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "gene_chemical_metabolism.tsv", session, all_cands, all_sents, header_str, 'gene_cid', 'chemical_cid')

  1%|▏         | 51/3452 [00:00<00:06, 495.51it/s]

Clearing existing...
Running UDF...


100%|██████████| 3452/3452 [00:05<00:00, 607.69it/s]


Number of candidates: 3148
[LSTM] Loaded model <gene_chemical_metabolism.lstm>
Loading all candidates from db...
3148 candidates load from db!
Applying LSTM to candidates...
Saved 3148 marginals
CPU times: user 17.9 s, sys: 748 ms, total: 18.7 s
Wall time: 7.03 s
LSTM applied!
Storing candidate labels into result file: results/simcyp298/gene_chemical_metabolism.tsv
Amount of candidates: 3148
Load mariginals from db...
Marginals loaded!
Building sentence to document map...
Map built!
Saved 521 positive predicitions for binary relation!
CPU times: user 3.14 s, sys: 64 ms, total: 3.2 s
Wall time: 2.91 s


In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor



ChemicalGeneInhibition = candidate_subclass('ChemicalGeneInhibition', ['chemical', 'gene'])
candidate_gene_chemical_inhibit_extractor = PretaggedCandidateExtractor(ChemicalGeneInhibition, ['Chemical', 'Gene'])


for k, sents in enumerate([all_sents]):
    candidate_gene_chemical_inhibit_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(ChemicalGeneInhibition).filter(ChemicalGeneInhibition.split == k).count())
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('chemical_gene_inhibition.lstm')

print("Loading all candidates from db...")
all_cands = session.query(ChemicalGeneInhibition).filter(ChemicalGeneInhibition.split == 0).order_by(ChemicalGeneInhibition.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','chemical_cid', 'chemical_span', 'gene_cid', 'gene_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "chemical_gene_inhibition.tsv", session, all_cands, all_sents, header_str, 'chemical_cid', 'gene_cid')